In [1]:
import os

In [2]:
%pwd

'd:\\End_to_End_ML_project_Chiken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End_to_End_ML_project_Chiken_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [19]:
%pip install py7zr

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 2.4 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.8 MB 1.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.8 MB 1.8 MB/s eta 0:00:01
   ---------------------------------- ----- 1.6/1.8 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 1.5 MB/s  0:00:01

  Attempting uninstall: brotli

    Found existing installation: Brotli 1.0.9

    Uninstalling Brotli-1.0.9:

   ---- -----------------------------------  1/10 [brotli]
   ---- -----------------------------------  1/10 [brotli]
   ---- -----------------------------------  1/10 [brotli]
   ---- -----------------------------------  1/10 [brotli]
   ---- -----------------------------------  1/10 [brotli]
   ---- -


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n+{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        Extracts the downloaded archive (zip or 7z) into the data directory
        Returns None; raises informative errors if extraction fails or dependencies are missing
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        file_path = self.config.local_data_file
        # Try zip extraction first
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                logger.info(f"Extracted zip archive to: {unzip_path}")
                return
        except zipfile.BadZipFile:
            logger.info("Not a zip archive, will attempt other extractors (e.g. 7z)")
        except Exception as e:
            logger.exception(f"Failed to extract as zip: {e}")
            raise e

        # Try 7z extraction using py7zr if available
        try:
            import py7zr
        except Exception:
            logger.error("py7zr is not installed; cannot extract .7z archives. Install with 'pip install py7zr' or change the source to a zip archive.")
            raise RuntimeError("py7zr not installed; cannot extract .7z archives.")

        try:
            with py7zr.SevenZipFile(file_path, mode='r') as archive:
                archive.extractall(path=unzip_path)
                logger.info(f"Extracted 7z archive to: {unzip_path}")
        except Exception as e:
            logger.exception(f"Failed to extract 7z archive: {e}")
            raise e

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-16 19:40:04,739: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-16 19:40:04,745: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-16 19:40:04,748: INFO: common: created directory at: artifacts]
[2025-12-16 19:40:04,749: INFO: common: created directory at: artifacts/data_ingestion]


[2025-12-16 19:40:13,472: INFO: 3291701381: artifacts/data_ingestion/data.7z download! with following info: 
+Connection: close
Content-Length: 11108677
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/octet-stream
ETag: "2481260637729293972966a2dccea991c6690738115de810b048bad071843995"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D735:2A538C:1A1D90:349BB5:6941683C
Accept-Ranges: bytes
Date: Tue, 16 Dec 2025 14:10:06 GMT
Via: 1.1 varnish
X-Served-By: cache-del21731-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1765894205.133890,VS0,VE887
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 89d7a5d14a60072b8ae6051d58f908a79abebe94
Expires: Tue, 16 Dec 2025 14:15:06 GMT
Source-Age: 1

]
[2025-12-16 19:40:13,508: INFO: 32917013